<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#py4DSTEM-io" data-toc-modified-id="py4DSTEM-io-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>py4DSTEM io</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Data" data-toc-modified-id="Data-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Data</a></span></li><li><span><a href="#Version-info" data-toc-modified-id="Version-info-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>Version info</a></span></li><li><span><a href="#Set-up" data-toc-modified-id="Set-up-1.0.3"><span class="toc-item-num">1.0.3&nbsp;&nbsp;</span>Set up</a></span></li><li><span><a href="#Non-native-files:-loading-4D-datacubes" data-toc-modified-id="Non-native-files:-loading-4D-datacubes-1.0.4"><span class="toc-item-num">1.0.4&nbsp;&nbsp;</span>Non-native files: loading 4D datacubes</a></span></li><li><span><a href="#Native-HDF5-files:-browsing-and-loading-data" data-toc-modified-id="Native-HDF5-files:-browsing-and-loading-data-1.0.5"><span class="toc-item-num">1.0.5&nbsp;&nbsp;</span>Native HDF5 files: browsing and loading data</a></span></li><li><span><a href="#The-DataObject-class" data-toc-modified-id="The-DataObject-class-1.0.6"><span class="toc-item-num">1.0.6&nbsp;&nbsp;</span>The <code>DataObject</code> class</a></span></li><li><span><a href="#Constructing-DataObject-instances" data-toc-modified-id="Constructing-DataObject-instances-1.0.7"><span class="toc-item-num">1.0.7&nbsp;&nbsp;</span>Constructing <code>DataObject</code> instances</a></span></li><li><span><a href="#Native-files:-save,-append,-copy" data-toc-modified-id="Native-files:-save,-append,-copy-1.0.8"><span class="toc-item-num">1.0.8&nbsp;&nbsp;</span>Native files: save, append, copy</a></span></li><li><span><a href="#Native-files:-remove,-overwrite,-repack" data-toc-modified-id="Native-files:-remove,-overwrite,-repack-1.0.9"><span class="toc-item-num">1.0.9&nbsp;&nbsp;</span>Native files: remove, overwrite, repack</a></span></li><li><span><a href="#Metadata" data-toc-modified-id="Metadata-1.0.10"><span class="toc-item-num">1.0.10&nbsp;&nbsp;</span>Metadata</a></span></li><li><span><a href="#topgroup-and-heirarchical-formatting:-.h5-files-containing-multiple-py4DSTEM-'files'" data-toc-modified-id="topgroup-and-heirarchical-formatting:-.h5-files-containing-multiple-py4DSTEM-'files'-1.0.11"><span class="toc-item-num">1.0.11&nbsp;&nbsp;</span><code>topgroup</code> and heirarchical formatting: .h5 files containing multiple py4DSTEM 'files'</a></span></li></ul></li></ul></li></ul></div>

# py4DSTEM io

This notebook discusses / demonstrates the read/write functionality of the py4DSTEM package.

Use cases shown here include:
- Non-native files: loading 4D datacubes (supported: .dm3/.dm4)
- Native HDF5 files: browsing and loading data
- Native files: save, append, copy
- Native files: remove, overwrite, repack
- Metadata: read, write, append
- `topgroup` and heirarchical formatting: .h5 files containing multiple py4DSTEM 'files'

### Data
This notebooks creates a number of different test files.  They are all placed in a single directory which you'll need to specify.  To run this notebook locally, 
1. Create a directory to store this notebook's input and output files somewhere on your system. In the cell immediately below this one, set `dirpath` to point to this folder.
2. Download the sample .dm file, and the sample .h5 file. They can be found [here](https://drive.google.com/file/d/1B-xX3F65JcWzAg0v7f1aVwnawPIfb5_o/view?usp=sharing) and [here](https://drive.google.com/file/d/12Q3T57x9N2vkyY0llqBLKn_0JPurQM6Y/view?usp=sharing).  Put both files in the directory you made.  In the cell below, make sure `filename_dm` and `filename_py4DSTEM_sample` specify the names of these two files.

The experimental 4DSTEM data used in this notebook was collected by Steven Zeltmann.

### Version info

Last updated on 2019-11-25 with py4DSTEM version 0.11.2.

In [ ]:
# Set file and directory names
dirpath = "data/"     # Please set this
filename_dm = "small4DSTEMscan_10x10.dm3"                             # and this
filename_py4DSTEM_sample = "small4DSTEMscan_10x10.h5"                 # and this
filename_py4DSTEM_1 = "py4DSTEM_iotest_1.h5"
filename_py4DSTEM_2 = "py4DSTEM_iotest_2.h5"

### Set up

In [ ]:
# Imports
import numpy as np
import py4DSTEM
from file_getter import download_file_from_google_drive

In [ ]:
# Filepath handling
from pathlib import Path
from os.path import exists
from os import remove as rm, listdir

dpath = Path(dirpath)
filepath_dm = dpath/Path(filename_dm)
filepath_py4DSTEM_sample = dpath/Path(filename_py4DSTEM_sample)
filepath_py4DSTEM_1 = dpath/Path(filename_py4DSTEM_1)
filepath_py4DSTEM_2 = dpath/Path(filename_py4DSTEM_2)

if exists(filepath_dm):
    pass
else:
    download_file_from_google_drive(id_='1B-xX3F65JcWzAg0v7f1aVwnawPIfb5_o' ,
                                    destination=f'{filepath_dm}')
if exists(filepath_py4DSTEM_sample):
    pass
else:
    download_file_from_google_drive(id_='12Q3T57x9N2vkyY0llqBLKn_0JPurQM6Y',
                                   destination=f'{filepath_py4DSTEM_sample}')
    


assert(exists(dpath)), "The specified directory {} does not exist".format(dpath)
assert(exists(filepath_dm)), "The specified .dm file {} does not exist".format(filepath_dm)

# Utility functions for clearing files
def rm_file(fp):
    if exists(fp):
        rm(fp)
rm_file(filepath_py4DSTEM_1)
rm_file(filepath_py4DSTEM_2)

### Non-native files: loading 4D datacubes
from 
- .dm3/.dm4
- numpy array
- empad TK
- medipix TK
- 4DCAMERA TK

In [ ]:
# Load data from a .dm file
datacube = py4DSTEM.io.read(filepath_dm)

In [ ]:
# The output is an instance of the py4DSTEM DataCube class
datacube

In [ ]:
# The data
datacube.data

In [ ]:
# The data shape after reading the file
print(datacube.data.shape)
print((datacube.R_Nx,datacube.R_Ny,datacube.Q_Nx,datacube.Q_Ny))

In [ ]:
# For some files, the scan shape is not in the file metadata, and needs to be set manually
datacube.set_scan_shape(10,10)
print(datacube.data.shape)
print((datacube.R_Nx,datacube.R_Ny,datacube.Q_Nx,datacube.Q_Ny))

In [ ]:
# Preliminary data visualization - the maximum diffraction pattern
max_dp = np.max(datacube.data, axis=(0,1))
py4DSTEM.visualize.show(max_dp,0,2)

In [ ]:
# If you can get your 4D datacube into a numpy array,
# you can create a DataCube directly
data4d = np.ones((5,5,6,6))
datacube_fromarray = py4DSTEM.io.DataCube(data=data4d)
datacube_fromarray

### Native HDF5 files: browsing and loading data

In [ ]:
# Display file contents, without loading anything
py4DSTEM.io.read(filepath_py4DSTEM_sample)

In [ ]:
# Print file version
v = py4DSTEM.io.native.get_py4DSTEM_version(filepath_py4DSTEM_sample)
print("written by py4DSTEM version {}.{}.{}".format(v[0],v[1],v[2]))

In [ ]:
# Load some named datablocks
datacube = py4DSTEM.io.read(filepath_py4DSTEM_sample, data_id='4ddatacube')
max_dp = py4DSTEM.io.read(filepath_py4DSTEM_sample, data_id='max_dp')
three_dps = py4DSTEM.io.read(filepath_py4DSTEM_sample, data_id='three_dps')
BF_image = py4DSTEM.io.read(filepath_py4DSTEM_sample, data_id='BF_image')
some_bragg_disks = py4DSTEM.io.read(filepath_py4DSTEM_sample, data_id='some_bragg_disks')
braggdisks = py4DSTEM.io.read(filepath_py4DSTEM_sample, data_id='braggpeaks')

### The `DataObject` class
py4DSTEM reads to and writes from chunks of data stored as instances of a class called `DataObject`.  `DataObject` has seven child classes: `DataCube`, `CountedDataCube`, `RealSlice`, `DiffractionSlice`, `PointList`, `PointListArray`, and `Metadata`.  The first six are discussed here.  Storing and retrieving metadata and the `Metadata` class are discussed later in this notebook.

In [ ]:
# DataCubes are for 4D-STEM scans - they contain 2D grids of 2D diffraction patterns
datacube

In [ ]:
assert(isinstance(datacube,py4DSTEM.io.DataObject))
assert(isinstance(datacube,py4DSTEM.io.DataCube))

In [ ]:
datacube.data.shape

In [ ]:
# DiffractionSlices and RealSlices
# These are intended as containers for data that is 2D, either
# in the shape of diffraction or real space.
max_dp

In [ ]:
max_dp.data.shape

In [ ]:
py4DSTEM.visualize.show(max_dp.data,0,2,figsize=(6,6))

In [ ]:
BF_image

In [ ]:
BF_image.data.shape

In [ ]:
py4DSTEM.visualize.show(BF_image.data,contrast='minmax',figsize=(6,6))

In [ ]:
# They can also store 3D data, corresponding to stacks of 2D images
three_dps

In [ ]:
three_dps.data.shape

In [ ]:
py4DSTEM.visualize.show_image_grid(lambda i:three_dps.data[:,:,i],1,3,min=0.5,max=2,axsize=(5,5))

In [ ]:
# PointList
some_bragg_disks

In [ ]:
# This is essentially a wrapper / .h5 interface for numpy structured arrays [https://numpy.org/doc/stable/user/basics.rec.html]
some_bragg_disks.data

In [ ]:
some_bragg_disks.data['qx']

In [ ]:
# PointListArray
braggdisks

In [ ]:
# These store a PointList at every scan position
braggdisks.shape

In [ ]:
# Retrieve a PointList
braggdisks.get_pointlist(4,4)

In [ ]:
# Show the data in one of the PointLists
braggdisks.get_pointlist(4,4).data

### Constructing `DataObject` instances
Saving information to a py4DSTEM file normally involves first saving the data as an instance of one of the `DataObject` child classes, then passing those to the `save` or `append` function.

In [ ]:
# Generate some data to save

# Bright-field image
x0,y0,R = 121,136,25
py4DSTEM.visualize.show_circ(max_dp.data,0,2,center=(x0,y0),R=R,alpha=0.25,figsize=(6,6))
BF_image_array = py4DSTEM.process.virtualimage.get_virtualimage_circ(datacube,x0,y0,R)
py4DSTEM.visualize.show(BF_image_array,contrast='minmax',figsize=(6,6))

# Dark-field image
x0,y0,R = 102,97,15
py4DSTEM.visualize.show_circ(max_dp.data,0,2,center=(x0,y0),R=R,alpha=0.25,figsize=(6,6))
DF_image_array = py4DSTEM.process.virtualimage.get_virtualimage_circ(datacube,x0,y0,R)
py4DSTEM.visualize.show(DF_image_array,contrast='minmax',figsize=(6,6))

# Annular dark-field image
x0,y0,Ri,Ro = 121,136,40,80
py4DSTEM.visualize.show_annuli(max_dp.data,0,2,center=(x0,y0),Ri=Ri,Ro=Ro,alpha=0.25,figsize=(6,6))
ADF_image_array = py4DSTEM.process.virtualimage.get_virtualimage_ann(datacube,x0,y0,Ri,Ro)
py4DSTEM.visualize.show(ADF_image_array,contrast='minmax',figsize=(6,6))

In [ ]:
# Make a RealSlice
BF_image_realslice = py4DSTEM.io.RealSlice(data=BF_image_array,name='BF_image')

In [ ]:
BF_image_realslice.data.shape

In [ ]:
# Make a RealSlice with several 2D arrays
images_realslice = py4DSTEM.io.RealSlice(
                data=np.dstack([BF_image_array,DF_image_array,ADF_image_array]),
                name='virtual_images',
                slicelabels=['BF','DF','ADF'])

In [ ]:
images_realslice.data.shape

In [ ]:
# Individual 2D arrays can be accessed by slicing into the array directly
DF_image_retrievedByIndex = images_realslice.data[:,:,1]

py4DSTEM.visualize.show(DF_image_retrievedByIndex,contrast='minmax',figsize=(6,6))

In [ ]:
# Individual 2D arrays can also be accessed by name
images_realslice.slicelabels

In [ ]:
DF_image_retrievedByName = images_realslice.slices['DF']

py4DSTEM.visualize.show(DF_image_retrievedByName,contrast='minmax',figsize=(6,6))

In [ ]:
# The two arrays we just retrieved are the same...
assert(np.sum(images_realslice.data[:,:,1]-images_realslice.slices['DF'])==0)

In [ ]:
# Caution!
# Note that RealSlice.slices (or DiffractionSlice.slices) points to 2D slices of RealSlice.data,
# and should only be used to retrieve data, *not* to assign data.  Assignment of .slices will
# not change the .data attribute, leading to inconsistencies - e.g.:

#images_realslice.slices['DF'] = 0                            # Comment the next line and uncomment this one to break the assert statement
images_realslice.slices['DF'] = images_realslice.data[:,:,1]  # Comment the previous line and uncomment this one to pass the assert statement
assert(np.sum(images_realslice.data[:,:,1]-images_realslice.slices['DF'])==0)

In [ ]:
# Naming dataobjects isn't strictly necessary, but is important - these are how
# the datablock will be identified when saved to an .h5 file.
images_realslice.name

In [ ]:
# DiffractionSlice has the same interface as RealSlice
three_dps = py4DSTEM.io.DiffractionSlice(
                 data=np.dstack([datacube.data[3,3+i,:,:] for i in range(3)]),
                 slicelabels=['dp1','dp2','dp3'],
                 name='three_dps')

In [ ]:
# PointList is intended to be flexible, enabling storage of N-points, each in M-dimensions.
# As an example, let's save a length 500 array of 2D points representing the
# functions x and cos(x)
N = 500
x = np.linspace(0,2*np.pi,N)
coords = [('x',float),('cos(x)',float)]
data = np.zeros(N,dtype=coords)              # Create a numpy structured array
data['x'] = x                                # populate the array
data['cos(x)'] = np.cos(x)
cosine_curve = py4DSTEM.io.PointList(coordinates=coords,data=data,name='cosine')

In [ ]:
cosine_curve

In [ ]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots()
ax.plot(cosine_curve.data['x'],cosine_curve.data['cos(x)'])
plt.show()

In [ ]:
# CountedDataCube, and PointListArrays are less likely to need to be generated
# from scratch.  They may be outputs of py4DSTEM functions, e.g. fing_bragg_disks returns
# a PointListArray of Bragg peak positions.

### Native files: save, append, copy

In [ ]:
# If the files we'll save in this section already exist, remove them 
rm_file(filepath_py4DSTEM_1)
rm_file(filepath_py4DSTEM_2)

In [ ]:
# Save a new file containing a single dataobject
py4DSTEM.io.save(filepath_py4DSTEM_1, data=max_dp)

In [ ]:
py4DSTEM.io.read(filepath_py4DSTEM_1)

In [ ]:
# Append more dataobjects to this file
py4DSTEM.io.append(filepath_py4DSTEM_1, data=BF_image)                       # We can append data one object at a time
py4DSTEM.io.append(filepath_py4DSTEM_1, data=[three_dps,some_bragg_disks])   # or we can append a list of DataObjects all at once

In [ ]:
py4DSTEM.io.read(filepath_py4DSTEM_1)

In [ ]:
# If we try to save a file where one already exists with the 'save' function
# we'll get an error message
py4DSTEM.io.save(filepath_py4DSTEM_1, data=[max_dp,BF_image])

In [ ]:
# If we're sure we don't need the old file, we can overwrite it using the 'overwrite' argument
py4DSTEM.io.save(filepath_py4DSTEM_1, data=[max_dp,BF_image], overwrite=True)

In [ ]:
py4DSTEM.io.read(filepath_py4DSTEM_1)

In [ ]:
# Copy a file
py4DSTEM.io.copy(filepath_py4DSTEM_1,filepath_py4DSTEM_2)

In [ ]:
py4DSTEM.io.read(filepath_py4DSTEM_2)

### Native files: remove, overwrite, repack

When removing or overwriting data blocks from an .h5 file, there's generally two options:

1. remove the object from user space, such that the object no longer appears when we print the file contents, it's name is now free, and a new object of this name can be saved.  However, the disk space taken up by this object has not been freed, and is still taken up by this file.  This may be fine for some datablocks (i.e. a single 2D array) but less desireable for large data blocks.

2. completly remove an object, such that the associated disk space is released, and the file size is accordingly reduced.  This requires re-writing all the *other* contents to a new file, then deleting the original file, therefore may be slow for larger files.

When removing an object at index `n` from a file, option 1 is accomplished with

```
py4DSTEM.io.remove(filepath, data=n, delete=False)
```
and option 2 is accomplished with
```
py4DSTEM.io.remove(filepath, data=n)
```
which is equivalent to 
```
py4DSTEM.io.remove(filepath, data=n)
py4DSTEM.io.repack(filepath)
```

Note that `repack` will only save py4DSTEM DataObjects - so if you have an .h5 file that you've added custom blocks of data to, they'll be lost if you run `repack` on this file!

In [ ]:
# Set up files
rm_file(filepath_py4DSTEM_1)
py4DSTEM.io.copy(filepath_py4DSTEM_sample,filepath_py4DSTEM_1)

In [ ]:
# Do a 'hard' remove, i.e. option 2
py4DSTEM.io.read(filepath_py4DSTEM_1)
py4DSTEM.io.remove(filepath_py4DSTEM_1, data=1)
py4DSTEM.io.read(filepath_py4DSTEM_1)

In [ ]:
# Overwriting an object

# Let's say we want to use a slightly large detector for our bright-field image
# We could save a new object called BF_image_2, but that approach can get confusing quickly!
# Instead, we'll overwrite the existing object called BF_image

# Compute a new bright-field image
x0,y0,R = 121,136,35
py4DSTEM.visualize.show_circ(max_dp.data,0,2,center=(x0,y0),R=R,alpha=0.25,figsize=(6,6))
BF_image_array_biggerDetector = py4DSTEM.process.virtualimage.get_virtualimage_circ(datacube,x0,y0,R)
py4DSTEM.visualize.show(BF_image_array,contrast='minmax',figsize=(6,6))

# Make a new RealSlice.  Note that the 'name' field is the same as the old BF RealSlice
BF_image_realslice_biggerDetector = py4DSTEM.io.RealSlice(data=BF_image_array_biggerDetector,name='BF_image')

In [ ]:
# When we pass this to append, we'll get an error message signaling there's already an object with this name
py4DSTEM.io.append(filepath_py4DSTEM_1, data=BF_image_realslice_biggerDetector)

In [ ]:
# A 'soft' overwrite, i.e. which does not release the disk space, is accomplished with 'overwrite=1'
# A 'hard' overwrite, i.e. which releases the disk space by re-writing the file, is accomplished with 'overwrite=2'
# Using 'overwrite=2' is equivalent to using 'overwrite=1' followed by calling repack(filepath)
py4DSTEM.io.append(filepath_py4DSTEM_1, data=BF_image_realslice, overwrite=2)

In [ ]:
py4DSTEM.io.read(filepath_py4DSTEM_1)

### Metadata

In [ ]:
# Set up files
rm_file(filepath_py4DSTEM_1)
rm_file(filepath_py4DSTEM_2)

In [ ]:
# When reading a non-native datacube, a Metadata instance is generated
# and is attached as an attribute to the datacube
datacube = py4DSTEM.io.read(filepath_dm)
datacube.metadata

In [ ]:
# The metadata is organized into five dictionaries.  They are: 
# 'microscope', 'calibration', 'sample', 'user', 'comments'
# They're reserved for the following uses:
# 'microscope': everything from the raw / original file goes here.
# 'calibration': all calibrations added later by the user go here.
# 'sample': information about the sample and sample prep.
# 'user': information about the microscope operator who acquired the data,
#  as well as the user who performed the computational analysis.
# 'comments': general use space for any other information
datacube.metadata.dicts

In [ ]:
# Metadata can be accessed directly from the dictionaries
# or you can also get/set methods
print(datacube.metadata.microscope['R_pixel_size'])
print(datacube.metadata.get_R_pixel_size())

In [ ]:
# Metadata should be assigned using the set methods
datacube.metadata.set_R_pixel_size(2.0)
datacube.metadata.set_R_pixel_size_units('nm')
print(datacube.metadata.get_R_pixel_size())

In [ ]:
# Some metadata items may exist in two places - 'microscope'
# and 'calibration'.  For instance, this happens if the microscope had
# some pixel calibrations, and then during data processing the user
# re-performs more accurate calibrations.  When set_R_pixel_size was called
# above, those were placed in 'calibration'.
print(datacube.metadata.microscope)
print(datacube.metadata.calibration)
print('')
print(datacube.metadata.microscope['R_pixel_size'])
print(datacube.metadata.calibration['R_pixel_size'])

In [ ]:
# The get methods will default to returning the 'calibration' values
# if they're present, and 'microscope' values if not.  Additional methods
# exist to only retrieve these items from a specific dictionary.
print(datacube.metadata.get_R_pixel_size())
print(datacube.metadata.get_R_pixel_size__microscope())
print(datacube.metadata.get_R_pixel_size__calibration())

In [ ]:
print(datacube.metadata.get_Q_pixel_size())
print(datacube.metadata.get_Q_pixel_size__microscope())
print(datacube.metadata.get_Q_pixel_size__calibration()) # Should throw error - has not been set!

In [ ]:
# This approach allows us to keep all the original metadata, while also
# allowing for more refined calibrations.

In [ ]:
# Any time a datacube is saved to a new .h5 file, py4DSTEM checks to see if it has
# a metadata instance attached.  If it does, it's saved to the .h5 file's metadata group
py4DSTEM.io.save(filepath_py4DSTEM_1,datacube)

In [ ]:
# Retrieve metadata from the .h5 file
metadata = py4DSTEM.io.read(filepath_py4DSTEM_1,metadata=True)

In [ ]:
datacube.metadata.dicts

In [ ]:
metadata.dicts

In [ ]:
metadata.get_R_pixel_size()

In [ ]:
# Save a new .h5 file which contains only the metadata
py4DSTEM.io.save(filepath_py4DSTEM_2,metadata)

In [ ]:
# Update an existing file's metadata with append
metadata.set_R_pixel_size(10.0)
py4DSTEM.io.append(filepath_py4DSTEM_2,metadata)
print(py4DSTEM.io.read(filepath_py4DSTEM_2,metadata=True).get_R_pixel_size())

In [ ]:
py4DSTEM.io.read(filepath_py4DSTEM_2)      # TODO include metadata when printing file contents

### `topgroup` and heirarchical formatting: .h5 files containing multiple py4DSTEM 'files'

In [ ]:
# writing two topgroups to a single .h5 file
rm_file(filename_py4DSTEM_2)
py4DSTEM.io.save(filename_py4DSTEM_2, topgroup='4DSTEM_dataset1', data=datacube)
py4DSTEM.io.save(filename_py4DSTEM_2, topgroup='4DSTEM_dataset2', data=three_dps)

In [ ]:
# reading: if the file has multiple topgroups, print their names with read()
py4DSTEM.io.read(filename_py4DSTEM_2)

In [ ]:
# reading: specifying a topgroup will list the contents of that subfile
py4DSTEM.io.read(filename_py4DSTEM_2,topgroup='4DSTEM_dataset2')

In [ ]:
# reading: specifying a topgroup and a valid data_id retreives a DataObject
data_from_topgroup = py4DSTEM.io.read(filename_py4DSTEM_2,
                                      topgroup='4DSTEM_dataset2',
                                      data_id='three_dps')
data_from_topgroup

In [ ]:
# appending: if the topgroup isn't specified, the topgroup names are printed
py4DSTEM.io.append(filename_py4DSTEM_2,data=BF_image_realslice)

In [ ]:
# appending
py4DSTEM.io.append(filename_py4DSTEM_2,
                   topgroup='4DSTEM_dataset2',
                   data=BF_image_realslice)
py4DSTEM.io.read(filename_py4DSTEM_2,topgroup='4DSTEM_dataset2')

In [ ]:
# Overwriting with multiple topgroups
# If there is an append conflict and the `overwrite` keyword is not specified,
# an error message is printed and nothing is appended
py4DSTEM.io.append(filename_py4DSTEM_2,
                   topgroup='4DSTEM_dataset2',
                   data=BF_image_realslice)

In [ ]:
# At this time overwriting objects in multi-topgroup files is possible only with overwrite=1 
py4DSTEM.io.append(filename_py4DSTEM_2,
                   topgroup='4DSTEM_dataset2',
                   data=BF_image_realslice,
                   overwrite=1)
py4DSTEM.io.read(filename_py4DSTEM_2,topgroup='4DSTEM_dataset2')

In [ ]:
# overwrite=2 fails 
py4DSTEM.io.append(filename_py4DSTEM_2,
                   topgroup='4DSTEM_dataset2',
                   data=BF_image_realslice,
                   overwrite=2)

In [ ]:
# Similarly, io.copy and io.repack are not yet supported for multi-topgroup files
# Use them at your peril!
# TODO: add assertions to prevent using these fns w/multi-tg files

In [ ]:
# Metadata and topgroups
md1 = py4DSTEM.io.read(filename_py4DSTEM_2,topgroup='4DSTEM_dataset1',metadata=True)
md2 = py4DSTEM.io.read(filename_py4DSTEM_2,topgroup='4DSTEM_dataset2',metadata=True)

In [ ]:
# Save a new piece of metadata to one of the topgroups and retrieve it again
md1.set_Q_pixel_size(100)
md2.set_Q_pixel_size(0.7)
py4DSTEM.io.append(filename_py4DSTEM_2,topgroup='4DSTEM_dataset1',data=md1)
py4DSTEM.io.append(filename_py4DSTEM_2,topgroup='4DSTEM_dataset2',data=md2)
md3 = py4DSTEM.io.read(filename_py4DSTEM_2,topgroup='4DSTEM_dataset1',metadata=True)
md4 = py4DSTEM.io.read(filename_py4DSTEM_2,topgroup='4DSTEM_dataset2',metadata=True)
print(md3.get_Q_pixel_size())
print(md4.get_Q_pixel_size())